In [6]:
from math import radians, sin, cos, sqrt, atan2
import numpy as np

def euclidean_distance(points1, points2):
    return np.linalg.norm(points1 - points2)

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calcular distância precisa entre dois pontos geográficos
    """
    R = 6371000  # Raio da Terra em metros
    
    # Converter para radianos
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    return R * c



lat1 = -20.27397
lon1 = -43.9878

lat2 = -20.27397
lon2 = -43.98779

points1 = np.array([lat1,lon1])
    
points2 = np.array([lat2,lon2])
    
distance = euclidean_distance(points1, points2) * 111000 
print(f"Euclidean Distance: {distance}") 
        



haversine_distance(lat1, lon1, lat2, lon2)

Euclidean Distance: 1.1100000003523292


1.0430600670878603

In [ ]:
def download_gpx_from_strava(link):
    """Baixa um arquivo GPX do Strava, dado o link da atividade."""
    try:
        export_link = f"{link}/export_gpx"
        response = requests.get(export_link)
        response.raise_for_status()  # Verifica se houve erro na requisição
        
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".gpx")
        with open(temp_file.name, "wb") as f:
            f.write(response.content)
        
        return temp_file.name
    except requests.exceptions.RequestException as e:
        st.error(f"Erro ao baixar o arquivo GPX: {e}")
        return None
    

    

Arquivo GPX salvo como arquivo.gpx.


In [ ]:
import requests
import re

# URL da página do Wikiloc



def get_position(url):
    # Configurar cabeçalhos para evitar bloqueio
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

    # Fazer a requisição
    response = requests.get(url, headers=headers)

    # Verificar se a página foi acessada com sucesso
    if response.status_code == 200:
        # Procurar pela variável "data" no HTML
        match = re.search(r"'data':'(.*?)'", response.text)
        if match:
            # Capturar os dados e dividir as posições
            data = match.group(1)
            positions = data.split(",")
            
            # Pegar a última posição (latitude e longitude)
            last_latitude = positions[-4]
            last_longitude = positions[-3]
            
            # print(f"Última posição: Latitude: {last_latitude}, Longitude: {last_longitude}")

            return last_latitude,last_longitude
        else:
            print("Dados de posição não encontrados.")
    else:
        print(f"Erro ao acessar a página. Código HTTP: {response.status_code}")



# url = "https://www.wikiloc.com/live/7436826/88EBULB"


get_position(url)

('-43.923468', '-19.949721')

In [26]:
from bs4 import BeautifulSoup
import json

def generate_map(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        script = soup.find('script', string=lambda t: t and 'initialActivity' in t)

        if script:
            json_text = script.string.strip()
            json_match = re.search(r'var initialActivity = ({.*?});', json_text, re.DOTALL)

            if json_match:
                json_text = json_match.group(1)
                json_text = json_text.replace("'", '"')

                json_data = json.loads(json_text)
                latlng = json_data.get('streams', {}).get('latlng', None)
                return latlng[-1]


generate_map("https://www.strava.com/beacon/3lAppyVROxB")

[-19.949774, -43.923447]

In [ ]:
from urllib.parse import urlparse
import folium
import folium.plugins

original_url = "https://livetrack.garmin.com/session/0a44556a-3c08-89f5-99b6-b9d57f5ba500/token/A13B7C18E49DF3C41C2504ED06614E2"


def get_position_garmin(url):
    parsed_url = urlparse(original_url)

    path_parts = parsed_url.path.split('/')
    session_id = path_parts[2]

    base_url = f"https://livetrack.garmin.com/services/session/{session_id}/trackpoints"

    response = requests.get(base_url)

    if response.status_code == 200:
        data = response.json()

        track_points = data.get("trackPoints", [])

        track_point = track_points[-1]
        
        coordinates = track_point["position"]["lat"], track_point["position"]["lon"]
    return coordinates

In [35]:
coordinates

(-19.97209008783102, -43.94257312640548)